In [10]:
# !pip install pytorch-nlp

     |████████████████████████████████| 90 kB 1.4 MB/s 


In [11]:
import sys
import numpy as np
import random as rn
import torch
# from pytorch_pretrained_bert import BertModel
from torch import nn
from torchnlp.datasets import imdb_dataset
# from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, BertModel, BertForMaskedLM

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

In [12]:
# import torchtext
# import torchtext.data
# # import torch.nn as nn
# # import torch.nn.functional as F

# from torchtext.vocab import Vocab

In [ ]:
# TEXT = torchtext.data.Field(lower=True, tokenize='spacy')
# Label = torchtext.data.LabelField(dtype = torch.float)

In [13]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

## Prepare the Data

In [14]:
train_data, test_data = imdb_dataset(train=True, test=True)
rn.shuffle(train_data)
rn.shuffle(test_data)
train_data = train_data[:1000]# only 1000 examples!!!
test_data = test_data[:100]

aclImdb_v1.tar.gz: 84.1MB [00:25, 3.28MB/s]


In [23]:
train_data[0]['text']

"This movie brings back many memories of the classic cinema of old, where actors didn't have to take their clothes off to make viewers watch their film.<br /><br />Firstly I think the main plus point of this movie is the amazing chemistry between Shahid and Amrita, it is definitely the making of the film.<br /><br />I have seen lots of comments regarding the film being sickly sweet and overly slushy. In response to this, I think to a certain degree this is a correct analysis, however considering this is a Barjatya film I think that compared to MPK, HAHK, HSSH and MPKDH, it has been toned down significantly. HSSH was almost unbearable to watch in some places.<br /><br />In this film however, when the sentimental moments come along, you find yourself smiling, wishing the budding couple all the best and hoping that nothing bad happens to them.<br /><br />Another major plus point is the performances of Shahid and Amrita. Both have acted very well, especially Shahid who looks great in the f

In [24]:
train_data[0]['sentiment']

'pos'

In [28]:
# zip(*map(lambda d: (d['text'], d['sentiment']), train_data)).iter()

AttributeError: 'zip' object has no attribute 'iter'

In [29]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(1000, 1000, 100, 100)

In [31]:
len(train_texts)

1000

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [33]:
tokenizer.tokenize('Hi my name is Dima')

['hi', 'my', 'name', 'is', 'dim', '##a']

In [37]:
gsa = '18. Governing Law: This Agreement shall be governed by and interpreted in accordance with the Federal laws of theUnited States, without reference to conflict-of-laws principles. If for any reason a court of competent jurisdiction finds any provision of this Agreement to be unenforceable, that provision will be enforced to the maximum extent possible to effectuate the intent of the parties, and the remainder of this Agreement will continue in full force and effect. This Agreement shall not be governed by the United Nations Convention on Contracts for the International Sale of Goods. Buyer agrees that exclusive jurisdiction for any dispute arising out of or relating to this Agreement lies within the venue mandated by applicable Federal law.'
# tokenizer.tokenize(gsa)

In [40]:
# gsa after BertTokenizer
tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], [gsa]))
train_token_id_gsa = list(map(tokenizer.convert_tokens_to_ids, tokens))
train_token_id_gsa

[[101,
  2324,
  1012,
  8677,
  2375,
  1024,
  2023,
  3820,
  4618,
  2022,
  9950,
  2011,
  1998,
  10009,
  1999,
  10388,
  2007,
  1996,
  2976,
  4277,
  1997,
  1996,
  19496,
  3064,
  2163,
  1010,
  2302,
  4431,
  2000,
  4736,
  1011,
  1997,
  1011,
  4277,
  6481,
  1012,
  2065,
  2005,
  2151,
  3114,
  1037,
  2457,
  1997,
  17824,
  7360,
  4858,
  2151,
  9347,
  1997,
  2023,
  3820,
  2000,
  2022,
  16655,
  2078,
  14821,
  3085,
  1010,
  2008,
  9347,
  2097,
  2022,
  16348,
  2000,
  1996,
  4555,
  6698,
  2825,
  2000,
  3466,
  20598,
  1996,
  7848,
  1997,
  1996,
  4243,
  1010,
  1998,
  1996,
  6893,
  1997,
  2023,
  3820,
  2097,
  3613,
  1999,
  2440,
  2486,
  1998,
  3466,
  1012,
  2023,
  3820,
  4618,
  2025,
  2022,
  9950,
  2011,
  1996,
  2142,
  3741,
  4680,
  2006,
  8311,
  2005,
  1996,
  2248,
  5096,
  1997,
  5350,
  1012,
  17634,
  10217,
  2008,
  7262,
  7360,
  2005,
  2151,
  7593,
  17707,
  2041,
  1997,
  2030,
  8800

In [42]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)                   
                   

(1000, 100)

In [43]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((1000, 512), (100, 512))

In [44]:
train_y = np.array(train_labels) == 'pos'
test_y = np.array(test_labels) == 'pos'
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((1000,), (100,), 0.489, 0.5)

In [49]:
# [[float(i > 0) for i in ii] for ii in train_tokens_ids]

In [51]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

# Baseline

In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [57]:
baseline_model = make_pipeline(TfidfVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)

In [58]:
baseline_predicted = baseline_model.predict(test_texts)

In [59]:
print(classification_report(test_labels, baseline_predicted))

precision    recall  f1-score   support

         neg       0.75      0.90      0.82        50
         pos       0.88      0.70      0.78        50

    accuracy                           0.80       100
   macro avg       0.81      0.80      0.80       100
weighted avg       0.81      0.80      0.80       100



# Bert Model

In [70]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        # _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        _, pooled_output = self.bert(tokens, attention_mask=masks)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
        

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [63]:
# str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [72]:
bert_clf = BertBinaryClassifier()
# bert_clf = bert_clf.cuda()


In [0]:
# str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

In [68]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
# y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
y, pooled = bert_clf.bert(x)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [73]:
y = bert_clf(x)
y.cpu().detach().numpy()

array([[0.5170465],
       [0.5997109],
       [0.524293 ]], dtype=float32)

In [0]:
# str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'6698.214912M'

In [74]:
# y, x, pooled = None, None, None
# torch.cuda.empty_cache()
# str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

# Fine-tune BERT

In [87]:
BATCH_SIZE = 16
EPOCHS = 1

In [ ]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

# str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


In [ ]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [ ]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [ ]:
#  torch.cuda.empty_cache()

In [ ]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        # print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
        

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
    

In [ ]:
np.mean(bert_predicted)

In [ ]:
print(classification_report(test_y, bert_predicted))

In [1]:
# get all 12 hidden states
# source: https://github.com/huggingface/transformers/issues/1827
from transformers import BertModel, BertConfig

config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)
model = BertModel.from_pretrained("bert-base-uncased", config=config)

outputs = model(inputs)
print(len(outputs))  # 3

hidden_states = outputs[2]
print(len(hidden_states))  # 13

embedding_output = hidden_states[0]
attention_hidden_states = hidden_states[1:]

NameError: name 'inputs' is not defined